In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nn-result/nn_result.csv
/kaggle/input/janestreet/data.parquet


This notebook's approach is using LSTM for times-series method.

* Ver 6: timestep = 1
* Ver 7: timestep = 2
* Ver 8: timestep = 3
* Ver 9: timestep = 5
* Ver 10: timestep = 4
* Ver 11: timestep = 1 + PCA
* Ver 12: timestep = 2 + PCA
* Ver 13: timestep = 3 + PCA
* Ver 14: timestep = 4 + PCA
* Ver 15: timestep = 5 + PCA
* Ver 16: timestep = 2 + NN ensemble

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings ("ignore")
import gc  

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
SEED = 1111
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [3]:
# this data is already excluded day <= 85

data = pd.read_parquet('../input/janestreet/data.parquet')

In [4]:
# Select trade with weight !=  0:
data = data[data['weight'] != 0]
# # limit memory use: we change datatype from float64 to float32
data = data.astype({c: np.float32 for c in data.select_dtypes(include='float64').columns}) 

# create target variable
data['action'] = (data['resp'] > 0)*1

# fill null values with mean of each feature
data.fillna(data.mean(),inplace=True)

#create fetures list
features = [c for c in data.columns if 'feature' in c]

In [5]:
# 20 / 80 split
df_trainvalid, df_test = np.split(data, [int(.8*len(data))])

df_train, df_valid = np.split(df_trainvalid, [int(.9*len(df_trainvalid))])

X_train = df_train[features]
y_train = df_train['action']

X_valid = df_valid[features]
y_valid = df_valid['action']

X_test = df_test[features]
y_test = df_test['action']


In [6]:
NORMALIZE_NONE = 0
NORMALIZE_MIN_MAX = 1
NORMALIZE_MEAN = 2


In [7]:
def normalize_data(df):
    if NORMALIZE_TYPE == NORMALIZE_MIN_MAX:
        return (df-df.min())/(df.max()-df.min())
    elif NORMALIZE_TYPE == NORMALIZE_MEAN:
        return (df-df.mean())/df.std()
    else:
        return df;


In [8]:
NORMALIZE_TYPE = NORMALIZE_MEAN

X_train = normalize_data(X_train)
X_valid = normalize_data(X_valid)
X_test = normalize_data(X_test)


In [9]:
def create_windows(data, window_shape, step = 1, start_id = None, end_id = None):
    
    data = np.asarray(data)
    data = data.reshape(-1,1) if np.prod(data.shape) == max(data.shape) else data
        
    start_id = 0 if start_id is None else start_id
    end_id = data.shape[0] if end_id is None else end_id
    
    data = data[int(start_id):int(end_id),:]
    window_shape = (int(window_shape), data.shape[-1])
    step = (int(step),) * data.ndim
    slices = tuple(slice(None, None, st) for st in step)
    indexing_strides = data[slices].strides
    win_indices_shape = ((np.array(data.shape) - window_shape) // step) + 1
    
    new_shape = tuple(list(win_indices_shape) + list(window_shape))
    strides = tuple(list(indexing_strides) + list(data.strides))
    
    window_data = np.lib.stride_tricks.as_strided(data, shape=new_shape, strides=strides)
    
    return np.squeeze(window_data, 1)

In [10]:
look_back = 2
look_ahead = 1
# LSTM expects 3D input (examples, timestep, features)
X_train_reshaped = create_windows(X_train, window_shape=look_back).astype(np.float32)
y_train_reshaped = create_windows(y_train, window_shape = look_ahead, start_id = look_back-1)

X_valid_reshaped = create_windows(X_valid, window_shape=look_back).astype(np.float32)
y_valid_reshaped = create_windows(y_valid, window_shape = look_ahead, start_id = look_back-1)

X_test_reshaped = create_windows(X_test, window_shape=look_back).astype(np.float32)
y_test_reshaped = create_windows(y_test, window_shape = look_ahead, start_id = look_back-1)

print(X_train_reshaped.shape, y_train_reshaped.shape)

(1131417, 2, 130) (1131417, 1, 1)


In [11]:
del data, df_train, df_valid, df_trainvalid, X_train, X_valid, y_train, y_valid
gc.collect()

62

In [12]:
batch_size = 4096 
step_train = len(X_train_reshaped)//batch_size
step_valid = len(X_valid_reshaped)//batch_size
step_test = len(X_test_reshaped)//batch_size


# # Create train, valid, test data with batch size 
train = tf.data.Dataset.from_tensor_slices((X_train_reshaped, y_train_reshaped)).batch(batch_size).repeat()
val = tf.data.Dataset.from_tensor_slices((X_valid_reshaped, y_valid_reshaped)).batch(batch_size).repeat()
test = tf.data.Dataset.from_tensor_slices((X_test_reshaped, y_test_reshaped)).batch(batch_size).repeat()


In [13]:
del X_train_reshaped, y_train_reshaped, X_valid_reshaped, y_valid_reshaped, y_test_reshaped
gc.collect()

20

In [14]:
l_r = 0.0001

# A Sequential model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(look_back,130),return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate = l_r),metrics=["AUC"])

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 2, 64)             49920     
_________________________________________________________________
dropout (Dropout)            (None, 2, 64)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
batch_normalization (BatchNo (None, 64)                256       
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 83,265
Trainable params: 83,137
Non-trainable params: 128
__________________________________________________

In [15]:
gc.collect()

7218

In [16]:
model.fit(train, validation_data=val, epochs=25, steps_per_epoch = step_train, validation_steps = step_valid, verbose = 1,
          callbacks = [EarlyStopping(monitor='val_loss', verbose=1, patience=10)])

Epoch 1/25
276/276 [==============================] - 11s 19ms/step - loss: 0.7613 - auc: 0.5030 - val_loss: 0.6934 - val_auc: 0.5066
Epoch 2/25
276/276 [==============================] - 4s 14ms/step - loss: 0.7213 - auc: 0.5045 - val_loss: 0.6935 - val_auc: 0.5100
Epoch 3/25
276/276 [==============================] - 4s 15ms/step - loss: 0.7141 - auc: 0.5051 - val_loss: 0.6937 - val_auc: 0.5101
Epoch 4/25
276/276 [==============================] - 4s 16ms/step - loss: 0.7091 - auc: 0.5053 - val_loss: 0.6935 - val_auc: 0.5090
Epoch 5/25
276/276 [==============================] - 4s 14ms/step - loss: 0.7052 - auc: 0.5064 - val_loss: 0.6935 - val_auc: 0.5083
Epoch 6/25
276/276 [==============================] - 4s 15ms/step - loss: 0.7028 - auc: 0.5069 - val_loss: 0.6936 - val_auc: 0.5087
Epoch 7/25
276/276 [==============================] - 4s 15ms/step - loss: 0.7010 - auc: 0.5063 - val_loss: 0.6935 - val_auc: 0.5091
Epoch 8/25
276/276 [==============================] - 4s 16ms/step -

In [17]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(test, steps = step_test)

print("test loss, test auc:", results)

Evaluate on test data
76/76 [==============================] - 1s 10ms/step - loss: 0.6914 - auc: 0.5314
test loss, test auc: [0.6914218068122864, 0.5313917994499207]


# Ensemble

In [18]:
# predict_proba for class 1 of lstm_model

prediction_proba_lstm = model.predict_proba(X_test_reshaped)
prediction_proba_lstm = prediction_proba_lstm.reshape(-1)
prediction_proba_lstm[:5]

array([0.48662037, 0.5321942 , 0.5097191 , 0.5041978 , 0.4837578 ],
      dtype=float32)

In [19]:
# predict_proba for class 1 of nn_model

prediction_proba_nn = pd.read_csv('../input/nn-result/nn_result.csv', )
prediction_proba_nn['0'].values[1:][:5]

array([0.47472417, 0.5421281 , 0.4973716 , 0.5151336 , 0.46307576])

In [20]:
# We will perform soft voting:
ensemble_results = (prediction_proba_lstm + prediction_proba_nn['0'].values[1:])/2
ensemble_results[:5]


array([0.48067227, 0.53716115, 0.50354534, 0.50966569, 0.47341678])

In [21]:
class_ensemble_results = [1 if i > 0.5 else 0 for i in ensemble_results]

# Utility score

In [22]:

result_df = pd.DataFrame({'Date': df_test['date'][look_back-1:], 'Weight': df_test['weight'][look_back-1:],
                          'Resp': df_test['resp'][look_back-1:], 'Action': class_ensemble_results})

result_df['P'] = result_df['Weight']*result_df['Resp']*result_df['Action']
result_df.head()

,Date,Weight,Resp,Action,P
1493257,430,11.982266,-0.004582,0,-0.000000
1493258,430,1.107787,-0.004491,1,-0.004975
1493259,430,1.312454,0.000542,1,0.000711
1493260,430,0.422074,-0.004623,1,-0.001951
1493261,430,2.849713,-0.011910,0,-0.000000


In [23]:
result_groupby_days = result_df[['Date', 'P']].groupby('Date').sum().reset_index()
print(result_groupby_days.shape)
result_groupby_days.head()

(70, 2)


,Date,P
0,430,-4.290695
1,431,-11.570710
2,432,-6.630291
3,433,-12.967695
4,434,7.240948


In [24]:
p = result_groupby_days['P'].values

t = (np.sum(p)/(np.sqrt(np.sum(p**2))))*np.sqrt(250/len(p))

u = min(max(t, 0), 6) * np.sum(p)

print(f"Utility score is: {u:.3f}")

Utility score is: 2897.839


In [25]:
result_df[['Date', 'Action']].to_csv(f'LSTM_result_timestep{look_back}.csv')